# Agentic & MCP Demo



## Setup

### Pre-Requisites

Before starting, ensure you have the following:
- A running Llama Stack server with the mcp::crm toolgroup configured

### Installing dependencies

This code requires `llama-stack` and the `llama-stack-client`, both at version `0.2.2`. Lets begin by installing them:

In [4]:
!pip3 install llama-stack-client==0.2.2 llama-stack==0.2.2

  Using cached pyaml-25.1.0-py3-none-any.whl.metadata (12 kB)
  Using cached termcolor-3.0.1-py3-none-any.whl.metadata (6.1 kB)
  Using cached blobfile-3.0.0-py3-none-any.whl.metadata (15 kB)
  Using cached fire-0.7.0.tar.gz (87 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached pycryptodomex-3.22.0-cp37-abi3-macosx_10_9_universal2.whl.metadata (3.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.3/273.3 kB 2.7 MB/s eta 0:00:002.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 5.4 MB/s eta 0:00:00m eta 0:00:010:01:01
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 4.0 MB/s eta 0:00:00m eta 0:00:010:01:01
Using cached blobfile-3.0.0-py3-none-any.whl (75 kB)
Using cached pyaml-25.1.0-py3-none-any.whl (26 kB)
Using cached termcolor-3.0

### Configuring logging

Now that we have our dependencies, lets setup logging for the application:

In [5]:
from llama_stack_client.lib.agents.event_logger import EventLogger
import logging

logger = logging.getLogger(__name__)
if not logger.hasHandlers():  
    logger.setLevel(logging.INFO)
    stream_handler = logging.StreamHandler()
    stream_handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(message)s')
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)

### Connecting to llama-stack server

For the llama-stack instance, you can either run it locally or connect to a remote llama-stack instance.


In [ ]:
import os
from dotenv import load_dotenv
from utils import get_any_available_model
load_dotenv()


base_url = os.getenv("REMOTE_BASE_URL", "http://localhost:5001")

from llama_stack_client import LlamaStackClient

client = LlamaStackClient(
    base_url=base_url
)
model = get_any_available_model(client)
    
logger.info(f"Connected to Llama Stack server @ {base_url} \n")

Connected to Llama Stack server @ http://localhost:5001 



### Validate tools are available in our llama-stack instance

When an instance of llama-stack is redeployed your tools need to re-registered. Also if a tool is already registered with a llama-stack instance, if you try to register one with the same `toolgroup_id`, llama-stack will throw you an error.

For this reason it is recommended to include some code to validate your tools and toolgroups. This is where the `mcp_url` comes into play. The following code will check that both the `builtin::websearch` and the `mcp::openshift` tools are registered as tools, but if the `mcp::openshift` tool is not listed there, it will attempt to register it using the mcp url.

If you are running the MCP server from source, the default value for this is: `http://localhost:8000/sse`.

If you are running the MCP server from a container, the default value for this is: `http://host.containers.internal:8000/sse`.

Make sure to pass the corresponding MCP URL for the server you are trying to register/validate tools for.

In [15]:


registered_tools = client.tools.list()
registered_toolgroups = [t.toolgroup_id for t in registered_tools]
logger.info(f"Your Llama Stack server is already registered with the following tool groups @ {set(registered_toolgroups)} \n")

Your Llama Stack server is already registered with the following tool groups @ {'mcp::crm', 'builtin::rag', 'builtin::websearch', 'builtin::code_interpreter', 'mcp::orders-service'} 



## Get active opportunities

### System Prompts for different models

**Note:** If you have multiple models configured with your Llama Stack server, you can choose which one to run your queries against. When switching to a different model, you may need to adjust the system prompt to align with that model’s expected behavior. Many models provide recommended system prompts for optimal and reliable outputs—these are typically documented on their respective websites.

In [10]:
# Here is a system prompt we have come up with which works well for this query

sys_prompt1= """You are a helpful assistant. Use tools to answer. When you use a tool always respond with a summary of the result."""

In [19]:
from llama_stack_client import Agent
# Create simple agent with tools
agent = Agent(
    client,
    model=model, # replace this with your choice of model
    instructions = sys_prompt1 , # update system prompt based on the model you are using
    tools=["mcp::crm"],
    tool_config={"tool_choice":"auto"}
)

user_prompts = ["Get one active opportunities"]
session_id = agent.create_session(session_name="OCP_demo")

for prompt in user_prompts:
    turn_response = agent.create_turn(
        messages=[
            {
                "role":"user",
                "content": prompt
            }
        ],
        session_id=session_id,
        stream=True,
    )
    for log in EventLogger().log(turn_response):
        log.print()

inference> 
tool_execution> Tool:getOpportunities Args:{'id': '1'}
tool_execution> Tool:getOpportunities Response:{"type":"text","text":"Active Opportunities: [\n  {\n    \"opportunity_id\": 1,\n    \"status\": \"active\",\n    \"account_id\": 1,\n    \"account_name\": \"Acme Corp\",\n    \"item_id\": 2,\n    \"description\": \"Upsell - Cloud package\",\n    \"amount\": \"5000.00\",\n    \"year\": 2025\n  },\n  {\n    \"opportunity_id\": 1,\n    \"status\": \"active\",\n    \"account_id\": 1,\n    \"account_name\": \"Acme Corp\",\n    \"item_id\": 1,\n    \"description\": \"Subscription renewal - Tier A\",\n    \"amount\": \"15000.00\",\n    \"year\": 2025\n  },\n  {\n    \"opportunity_id\": 2,\n    \"status\": \"active\",\n    \"account_id\": 2,\n    \"account_name\": \"Globex Inc\",\n    \"item_id\": 3,\n    \"description\": \"Enterprise license renewal\",\n    \"amount\": \"25000.00\",\n    \"year\": 2025\n  }\n]","annotations":null}
inference> The function "getOpportunities" return

### Output Analysis

Lets step through the output to further understands whats happening in this Agentic demo.

1. First the LLM sends off a tool call to the pods_run tool configured with the OpenShift MCP server, to run the pod with the requested docker image in the OpenShift cluster.
2. The tool successfully executes and creates the pod.
3. The LLM recieves the response from the tool call, results of the pod manifest created, along with the original query.
4. Finally, this context gets passed back to the LLM for the final inference. The inference result starts by responding to my initial question with some background, and then finally providing details about the pod specifications and configurations created as well mentionining that the pod might be in pending state indicating that it might take few minutes to successfully complete.

## Analyse account sentiment for active opportunities

Get a list of active opportunities, then find the associated support cases for each opportunity, then anayse the mood of the account.

In [ ]:
# Here is a system prompt we have come up with which works well for this query
sys_prompt2="""You are a helpful AI assistant, responsible for helping me find and communicate information back to my team.
    You have access to a number of tools.
    Whenever a tool is called, be sure return the Response in a friendly and helpful tone.
    When you are asked to find out about opportunities and accounts you must use a tool.
    """

In [31]:
from llama_stack_client.lib.agents.agent import Agent

agent = Agent(
    client=client,
    model=model,
    instructions=sys_prompt2,
    tools=["mcp::crm"],
    tool_config={"tool_choice":"auto"},
    sampling_params={
        "max_tokens":4096,
        "strategy": {"type": "greedy"},
    }
)

session_id = agent.create_session(session_name="Draft_email_with_latest_OCP_version")

prompts = [
    """Get one active opportunity""",
    """get a list of support cases for the associated account""",
    """Analyse the support cases and determine how happy the account is""",

]
for i, prompt in enumerate(prompts):    
    turn_response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=True,
    )
    logger.info(f"========= Turn: {i} =========")
    for log in EventLogger().log(turn_response):
        log.print()


========= Turn: 0 =========


inference> 
tool_execution> Tool:getOpportunities Args:{'id': '1'}
tool_execution> Tool:getOpportunities Response:{"type":"text","text":"Active Opportunities: [\n  {\n    \"opportunity_id\": 1,\n    \"status\": \"active\",\n    \"account_id\": 1,\n    \"account_name\": \"Acme Corp\",\n    \"item_id\": 2,\n    \"description\": \"Upsell - Cloud package\",\n    \"amount\": \"5000.00\",\n    \"year\": 2025\n  },\n  {\n    \"opportunity_id\": 1,\n    \"status\": \"active\",\n    \"account_id\": 1,\n    \"account_name\": \"Acme Corp\",\n    \"item_id\": 1,\n    \"description\": \"Subscription renewal - Tier A\",\n    \"amount\": \"15000.00\",\n    \"year\": 2025\n  },\n  {\n    \"opportunity_id\": 2,\n    \"status\": \"active\",\n    \"account_id\": 2,\n    \"account_name\": \"Globex Inc\",\n    \"item_id\": 3,\n    \"description\": \"Enterprise license renewal\",\n    \"amount\": \"25000.00\",\n    \"year\": 2025\n  }\n]","annotations":null}
inference> The active opportunity is:

* Opportun

========= Turn: 1 =========


inference> 
tool_execution> Tool:getSupportCases Args:{'id': '1'}
tool_execution> Tool:getSupportCases Response:{"type":"text","text":"Support Cases for account 1:[\n  {\n    \"case_id\": 1,\n    \"subject\": \"Login failure\",\n    \"description\": \"Customer unable to log in with correct credentials.\",\n    \"status\": \"open\",\n    \"severity\": \"High\",\n    \"created_at\": \"2025-04-16T22:57:50.901Z\",\n    \"account_name\": \"Acme Corp\"\n  },\n  {\n    \"case_id\": 2,\n    \"subject\": \"Slow dashboard\",\n    \"description\": \"Performance issues loading analytics dashboard.\",\n    \"status\": \"in progress\",\n    \"severity\": \"Critical\",\n    \"created_at\": \"2025-04-16T22:57:50.901Z\",\n    \"account_name\": \"Acme Corp\"\n  },\n  {\n    \"case_id\": 6,\n    \"subject\": \"Feature request: Dark mode\",\n    \"description\": \"Request to implement dark mode UI.\",\n    \"status\": \"closed\",\n    \"severity\": \"Low\",\n    \"created_at\": \"2025-04-16T22:57:50.901Z\

========= Turn: 2 =========


inference> Based on the support cases, I can analyze the account's satisfaction level. Here's my assessment:

1. **Login failure**: The customer is unable to log in with correct credentials, which indicates a significant issue with the account's functionality. This suggests that the account is not satisfied with the current state of the service.
2. **Slow dashboard**: The performance issues loading the analytics dashboard indicate that the account is experiencing technical difficulties, which can be frustrating and impact their ability to use the service effectively.
3. **Feature request: Dark mode**: While this case is closed, it suggests that the account is interested in seeing improvements to the service, specifically in terms of user experience. This indicates a level of dissatisfaction with the current state of the service.

Overall, based on these support cases, I would rate the account's satisfaction level as **Unhappy**. The account is experiencing significant technical issues 

#### Output Analysis



## Key Takeaways

This tutorial demonstrates how to build agentic MCP applications with Llama Stack. We do so by initializing an agent while giving it access to the MCP tools configured with Llama Stack, then invoking the agent on each of the specified queries. 